# Text from 10-K/Q Disclosures and Sections
## See list of available disclosures @ https://www.calcbench.com/disclosure_list
## Documentation @ http://calcbench.github.io/python_api_client/html/disclosures.html

In [ ]:
# %#pip install calcbench-api-client[BeautifulSoup,Backoff] tqdm

In [ ]:
import calcbench as cb
from calcbench.models.disclosure import FootnoteTypeTitle
from tqdm.notebook import tqdm
from itertools import product

cb.enable_backoff()

In [ ]:
tickers = cb.tickers(index="DJIA")

## Get individual disclosures

In [ ]:
disclosures_to_get = [
    "RiskFactors",
    "ManagementsDiscussionAndAnalysis",
]  # See the complete list @ https://www.calcbench.com/disclosure_list

In [ ]:
for ticker in tqdm(tickers, leave=False):
    docs = cb.disclosure_search(
        company_identifiers=[ticker],
        disclosure_names=disclosures_to_get,
        all_history=True,
        period_type="annual",
    )
    for document in docs:
        contents = document.get_contents_text()
        print(
            f"{ticker} {document.fiscal_year}-{document.fiscal_period} {document.disclosure_type_name}, Word Count {len(contents.split())}"
        )

## Get 10-K/Q sections and related disclosures by company and period

In [ ]:
years = range(2010, 2022)
periods = [0, 1, 2, 3]

In [ ]:
company_periods = list(product(tickers, years, periods))

In [ ]:
for ticker, fiscal_year, fiscal_period in tqdm(company_periods):
    disclosures = list(
        cb.disclosure_search(
            all_documents=True,
            company_identifiers=[ticker],
            year=fiscal_year,
            period=fiscal_period,
            use_fiscal_period=True,
        )
    )
    for d in (ticker_pbar:=tqdm(disclosures, desc=ticker, leave=False)):
        if d.footnote_type_title not in {
            FootnoteTypeTitle.TextBlock,
            FootnoteTypeTitle.PolicyTextBlock,
        }:
            text_contents = d.get_contents_text()
            word_count = len(text_contents.split(" "))
            ticker_pbar.set_description(f"{d} word count {word_count}")